In [ ]:
print('hello world')

# DataFrame

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import os
import yfinance

## 生成DF

### 生成方式

### dict -> df

In [ ]:
sample_dict = {
    'open' : [1, 2, 3, 4],
    'close' : [5, 6, 7, 8]
}

df = pd.DataFrame(sample_dict)

### list -> df

In [ ]:
sample_dict = [
    [1, 2, 3, 4],
    [5, 6, 7, 8]
]

df = pd.DataFrame(sample_dict, index = ['open', 'close'], columns = range(4))
df = df.T

## 基本操作

### 屬性 vs 方法
1.  屬性代表物件的 **靜態特徵、狀態或儲存** 的資料。它們就像名詞一樣，描述了物件的樣子。
 > 屬性是名詞，它們只是取回物件已經儲存好的資料。
 - 代表狀態或資料： 描述物件「是什麼」或「有什麼」。
 - 調用方式： 不需括號 ()。
 - 本質： 儲存在記憶體中的一個變數。
 > 程式碼概念 > 汽車實體,說明
 > 1. car.color,顏色,"描述汽車的顏色，資料是靜態的（例如：""紅色""）。"
 > 2. car.current_speed,目前時速,描述汽車當前的速度狀態，資料是可變的（例如：60）。
 > 3. df.shape,資料形狀,"描述 DataFrame 的尺寸（例如：(100, 5)）。"
 > 4. s.dtype,資料型態,描述 Series 中元素的資料型態。
---
2.  方法代表物件 **可以執行的動態行為、操作或計算** 。它們就像動詞一樣，描述了物件能做的事情。
 > 方法是動詞，它們需要被執行才能產生結果。
 - 代表動作或功能： 描述物件「能做什麼」。
 - 調用方式： 必須有括號 ()。括號的作用是告訴程式：「執行這個動作！」
 - 本質： 儲存在記憶體中的一個函數 (Function)。
 > 程式碼概念 > 汽車實體,說明
 > 1. car.accelerate(),加速,執行一個動作，讓速度這個屬性增加。
 > 2. car.brake(power),剎車,執行一個動作，可以接收參數 power 來決定減速的程度。
 > 3. df.mean(),計算平均值,執行計算所有欄位平均值的動作，並回傳結果。
 > 4. s.notna(),檢查非缺失值,執行檢查 Series 中所有值是否為非缺失值的動作，並回傳結果。


---

#### 常見屬性

In [ ]:
df.cumprod() # 累積乘法


## YahooFinance API套件
> pip install yfinance
> https://ranaroussi.github.io/yfinance/

In [ ]:
# 資料來源YahooFinance

import yfinance as yf
tw0050 = yf.download('0050.TW', start = '1993-01-01')

In [ ]:
tw0050.tail(1).round(2)

In [ ]:
tw0050.pct_change(periods = 1) * 100

In [ ]:
tw0050.info()

In [ ]:
# 遺漏值確認
tw0050[tw0050.isna()]

# 遺漏值刪除
tw0050.dropna(subset = [('Close', '0050.TW')])

check = tw0050.isna().any().any().sum() # 確認 沒有遺漏值

# 補上nan
tw0050.fillna(0)
tw0050.fillna(method = 'ffill', axis = 0) # 如果需要指定橫向/值向，才用method寫法，要不直接用tw0050.ffill()

In [ ]:
# 是否重複
ducon = tw0050[tw0050.duplicated()]
# display(ducon)
print(f'原有{len(tw0050)} 共有 {len(ducon)} 筆重複')

# 刪除重複值
# display(tw0050.drop_duplicates())
print(f'現在剩下 {len(tw0050)} 筆資料')

tw0050 = yf.download('0050.TW', start = '1993-01-01') # 恢復資料，等等繼續使用

In [ ]:
# column階層
'''
取得所有獨特值： 如果您只是想知道每個層級中包含了哪些唯一的標籤（類似字典的鍵），請使用 df.columns.levels。
取得所有欄位的標籤： 如果您想知道每個完整的欄位 Tuple 在特定層級（例如 Level 1）上的值是什麼，請使用 df.columns.get_level_values(層級編號或名稱)。
'''

tw0050.columns.levels # FrozenList([['Close', 'High', 'Low', 'Open', 'Volume'], ['0050.TW']])
tw0050.columns.get_level_values(1) # Index(['0050.TW', '0050.TW', '0050.TW', '0050.TW', '0050.TW'], dtype='object', name='Ticker')
tw0050.columns.get_level_values(0) # Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

In [ ]:
tw0050.sort_values(by = [(  'High', '0050.TW')], ascending = False)
tw0050[(  'pricerank', '0050.TW')] = tw0050[(  'High', '0050.TW')].rank(method = 'average', ascending = False).astype('int')
tw0050.sort_values(by = [(  'pricerank', '0050.TW')], ascending = False)

In [ ]:
# 切割
# 「小括號 () 和中括號 [] 在 qcut 結果中代表的差異」非常重要，這關係到區間的包含性 (Inclusivity)，也就是我們常說的開區間 (Open Interval) 和閉區間 (Closed Interval)。
tw0050[(  'Open_cut', '0050.TW')] = pd.cut(bins = [10, 20, 35, 40, 60, 100],x = tw0050[(  'Open', '0050.TW')])
tw0050[(  'Open_qcut', '0050.TW')] = pd.qcut(x = tw0050[(  'Open', '0050.TW')], q = 8)
tw0050


In [ ]:
tw0050 = yf.download('0050.TW', start = '1993-01-01') # 恢復資料，等等繼續使用
tw0050.columns = tw0050.columns.get_level_values(0)
tw0050.to_csv('test.csv', sep = ',', encoding = 'utf-8-sig')
tw0050.to_excel('test.xlsx', sheet_name = 'test')

df = pd.read_csv('test.csv', sep = ',', encoding = 'utf-8-sig')
df.set_index('Date', inplace = True)
dff = pd.read_excel('test.xlsx')
dff.set_index('Date', inplace = True)

display(df.head())
print('=============')
display(dff.head())



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (8, 6))
# plt.plot(df['Close'])
df.Close.plot(x = df.index)
plt.show()

In [ ]:
df[df.Volume != 0].Volume.plot.hist(bins = 30)
plt.show()
df[df.Volume != 0].diff(periods = 1).Volume.plot()
plt.show()

In [ ]:
# 成交量的增減對未來報酬影響
plt.rcParams['font.sans-serif'] = ['Noto Sans HK']
plt.rcParams['axes.unicode_minus'] = False # 確保負號正常顯示

df = df.drop_duplicates(keep = 'first')
df['未來的報酬'] = df['Close'].divide(df['Close'].shift(1)) - 1
df['成交量的增減'] = df['Volume'].diff(periods = 1)
df.loc['2015':].plot.scatter(x = '成交量的增減', y = '未來的報酬')
plt.legend(loc = 'upper center')

plt.show()

In [ ]:
df = yf.download('0050.TW', start = '1993-01-01') # 恢復資料，等等繼續使用
df.columns = df.columns.get_level_values(0)
df.loc['2025-11']['Volume'].plot.hist()
df[['Close', 'High', 'Low', 'Open']].plot.box()
plt.show()

In [ ]:
df = yf.download('0050.TW', start = '1993-01-01', auto_adjust = True) # 恢復資料，等等繼續使用
df.columns = df.columns.get_level_values(0)
df.groupby(df.index.year).agg(maxclose = ('Close', 'max'))


# 量增 or 量減
df['昨日成交量'] = df['Volume'].shift(1)
df['是否量增'] = df['Volume'] > df['昨日成交量']
df['未來報酬率'] = df['Close'].shift(-1) / df['Close'] - 1
df.groupby('是否量增').agg(因果 = ('未來報酬率', 'mean'))




In [ ]:
df = yf.download('0050.TW', start = '1993-01-01', auto_adjust = True) # 恢復資料，等等繼續使用
df.columns = df.columns.get_level_values(0)

df['量增減幅度'] = df['Volume'].diff()
df['量增減幅度分群'] = pd.qcut(df['量增減幅度'], q = 6)
df['昨日成交量'] = df['Volume'].shift(1)
df['是否量增'] = df['Volume'] > df['昨日成交量']
df['未來報酬率'] = df['Close'].shift(-1) / df['Close'] - 1
df.groupby('量增減幅度分群').agg(分群結果 = ('未來報酬率', 'mean'))

In [ ]:
df['依照量增幅度_報酬率'] = df.groupby('量增減幅度分群')['未來報酬率'].transform('mean')
df

In [ ]:
def sum1(a):
    return a + 3

df['Close_3'] = df['Close'].apply(sum1)
df

In [ ]:
df.to_excel('20251121.xlsx', sheet_name = 'test')
dff = pd.read_excel('20251121.xlsx')
dff.info()

In [ ]:
pd.to_datetime(dff['Date'])
dff.set_index('Date')

## 課堂練習
透過迴圈，將df資料逐一顯示

In [ ]:
for i in df.columns:
    for j in df.index:
        print(f'這個是 {i}的 第 {j}列', end = '\t')
        print(df.loc[j, i])

In [ ]:
for i in range(len(df.columns)):
    for j in range(len(df.index)):
        print(f'這個是 第 {i}行 第 {j}列的值 : ', end = '\t')
        print(df.iloc[j, i])

請實作出每日的  **開盤 及 收盤** 的平均價格

In [ ]:

def meanoc(df1 : pd.DataFrame, df2 : pd.DataFrame) -> pd.DataFrame:
    ans = (df1 + df2) / 2
    return ans

# temp_0050 = (tw0050['Close'] + tw0050['Open']) / 2 # 法一
temp_0050 = (tw0050[['Close', 'Open']].mean(axis = 1)).to_frame('aveP') # 法二
# temp_0050 = meanoc(tw0050['Close'], tw0050['Open']) # 法三
temp_0050.head()

In [ ]:
close5 = tw0050.iloc[-5:]['Close'].mean().round(2)
close5

In [ ]:
# 抓出成交量 > 平均數的日期

volmean = tw0050[('Volume', '0050.TW')].mean()
tw0050[(tw0050[('Volume', '0050.TW')].ge(volmean))]
